# Storing and Accessing objects via Descartes Labs Storage
__________________

The Descartes Labs Catalog API also supports arbitrary formats of data that don't fall neatly into either a raster or vector data model.

This guide covers the basic methods for generic blob storage. For a more in depth overview of all Catalog classes and their capabilities please visit the [API Reference](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) and [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html) sections in our Documentation page.

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Blob, properties as p

In [ ]:
import json

## Creating a Blob
The only required attribute for a [`Blob`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html) is a *unique* ID. Blobs may also contain various attributes to search and filter by, such as a geometry and [`tags`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.tags). 

In this first example we'll define a JSON dictionary of arbitrary field data and a geometry to go with it:

In [ ]:
# JSON of random field info
crop_info = {
    "crop": "hops",
    "acreage": 450,
}
# Geometry for the field
field_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-120.4023, 46.551],
            [-120.3859, 46.551],
            [-120.3859, 46.5534],
            [-120.4023, 46.5534],
        ]
    ],
}

#### _Note on Blobs_
We are using JSON for simplicity, however this could be _any object format_ you choose!

Let's start by creating a new blob named **hop_field_info**, and give it a list of tags to search by:

In [ ]:
blob = Blob(
    name="hop_field_info",
    tags=["examples"],
    geometry=field_geom
)
blob

### Uploading to a Blob from Data In Memory

Now that we have the data we want to store, we can add data to our blob and call [`Blob.upload_data()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.upload_data) to upload our dataset:

In [ ]:
blob.upload_data(json.dumps(crop_info))
blob

### Uploading Files as Blobs
If your object is located on disk, you can instead call [`Blob.upload()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.upload):

In [ ]:
# blob.upload("data/yakima.geojson")

#### Blob Attributes
Each saved blob has several attributes, a few of which are printed below:

In [ ]:
print("Blob ID:", blob.id)
print("Blob size:", blob.size_bytes)
print("Blob geometry: ", blob.geometry)
print("Blob assigned tags: ", blob.tags)
print("Blob ID: ", blob.id)

#### _Note on Namespaces_:
You'll note on the blob ID that, as with products, blobs also contain a _namespace_. A namespace defaults to the current user's organization and user hash if none is specified (such as with self-signup users). Therefore, the typical construction of a blob ID is as follows:

    data/my-org-id:my-user-id/blob-name
    
or

    data/my-user-id/blob-name

If you pass a string into the **namespace** argument, your blob ID will be:

    data/my-org-id:passed-namespace/blob-name
    

Where **data/** is the [`StorageType`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob.storage_type).

_Some users may not have an organization defined. Your namespace will simply be your user hash._

Similar to [02 Creating and Managing Products.ipynb](./02%20Creating%20and%20Managing%20Products.ipynb), we can recreate our default namespace with the current user's org and hash:

In [ ]:
##Setting namespace
auth = dl.auth.Auth.get_default_auth()
user_hash = auth.namespace
org = auth.payload['org']
default_namespace = f"{org}:{user_hash}" if org else user_hash

In [ ]:
print(blob.namespace==default_namespace)

## Searching Blobs
Catalog search methods can be performed across your storage objects, including geospatial searches as well. 

Below are examples of:
* Point intersection
* Polygon intersection
* Tag filter
* Filtering by Storage Type

In [ ]:
# Geospatial searches by intersection
## Intersect particular coordinate
print("Point Intersection:")
print(
    [
        b.id
        for b in Blob.search().intersects(
            {"type": "Point", "coordinates": [-120.40, 46.552]}
        )
    ]
)
print("Polygon Intersectsion:")
## Intersecting geometry object
print([b.id for b in Blob.search().intersects(field_geom)])
print("Tag Filter:")
# Filter by tags
print([b.id for b in Blob.search().filter(p.tags == "examples")])
print("Storage Type Filter:")
# Filter by Storage Type
print([b.id for b in Blob.search().filter(p.storage_type == "data").limit(10)])

We can also use a prefix filter to pick out these new blobs:

In [ ]:
for b in Blob.search().filter(p.name.prefix("foo/")):
    print(b.id)

*__Note__*: Only the name may contain internal '/' characters.

## Retrieving Data from Blobs
The blob data may be retrieved, either by downloading directly to a local file or some other file-like object (e.g. an io.IOBase object), or directly into memory. Here's a simple download to a file via [`Blob.download()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html?highlight=storagetype#descarteslabs.catalog.Blob.download):

In [ ]:
blob.download("data/data.json")

You can also download in raw bytes via [`Blob.data()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html?highlight=storagetype#descarteslabs.catalog.Blob.data):

In [ ]:
print(len(blob.data()))

## Sharing Blobs
As with products, you can add specific organizations or users as readers to your blobs to give others access. Simply update the readers list, then save the blob:

In [ ]:
# Adding coworker and org as readers
blob.readers = ["email:john.daily@gmail.com", "org:pga-tour"]
blob.save()
blob.readers

#### Note on Ownership
Blobs can also transfer ownership, however you cannot use the "email:" variant as an owner. Instead use the new owner's user hash:

In [ ]:
# blob.owners.append(f"user:{user_hash}")
# blob.save()

For more information visit our [Sharing Resources](https://docs.descarteslabs.com/guides/sharing.html) page.

## Deleting Blobs
Blobs can also be deleted by calling [`Blob.delete()`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html?highlight=storagetype#descarteslabs.catalog.Blob.delete):

In [ ]:
status = blob.delete()
if status:
    status.wait_for_completion()
blob

Cleaning up our workspace

In [ ]:
import os

os.remove("data/data.json")